In [28]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import string

def generate_synthetic_data(rows=1000, cols=30, seed=42):
    """
    Generates a synthetic DataFrame with mixed data types:
    - Boolean
    - Integer (numeric)
    - Float
    - Text
    - Categorical
    - Date

    Parameters:
    - rows: int, number of rows
    - cols: int, total number of columns

    Returns:
    - df: pd.DataFrame, synthetic dataset
    """
    np.random.seed(seed)
    random.seed(seed)

    col_types = ['bool', 'int', 'float', 'text', 'category', 'date']
    type_counts = {k: cols // len(col_types) for k in col_types}
    remainder = cols % len(col_types)
    for i in range(remainder):
        type_counts[col_types[i]] += 1

    data = {}

    for i in range(type_counts['bool']):
        data[f'bool_col_{i}'] = np.random.choice([True, False], size=rows)

    for i in range(type_counts['int']):
        data[f'int_col_{i}'] = np.random.randint(0, 1000, size=rows)

    for i in range(type_counts['float']):
        data[f'float_col_{i}'] = np.random.uniform(0, 1000, size=rows)

    for i in range(type_counts['text']):
        data[f'text_col_{i}'] = [''.join(random.choices(string.ascii_letters, k=10)) for _ in range(rows)]

    for i in range(type_counts['category']):
        data[f'cat_col_{i}'] = pd.Series(np.random.choice(['A', 'B', 'C', 'D'], size=rows)).astype("category")

    base_date = datetime(2020, 1, 1)
    for i in range(type_counts['date']):
        data[f'date_col_{i}'] = [base_date + timedelta(days=np.random.randint(0, 3650)) for _ in range(rows)]

    data[f'target'] = np.random.uniform(0, 1000, size=rows)

    df = pd.DataFrame(data)
    df['target'] = df['target'].astype(float)
    return df

# Generate and preview
main_df = generate_synthetic_data()
df = main_df.copy()
df.head()


,bool_col_0,bool_col_1,bool_col_2,bool_col_3,bool_col_4,int_col_0,int_col_1,int_col_2,int_col_3,int_col_4,...,cat_col_1,cat_col_2,cat_col_3,cat_col_4,date_col_0,date_col_1,date_col_2,date_col_3,date_col_4,target
0,True,False,True,False,False,946,839,945,801,955,...,C,D,B,B,2025-08-29,2020-05-08,2028-12-02,2025-11-30,2029-12-21,12.137458
1,False,True,False,False,True,686,123,70,588,599,...,A,D,A,A,2027-09-07,2024-01-10,2025-05-17,2026-10-08,2020-09-21,488.643897
2,True,True,False,False,True,750,992,363,809,755,...,C,A,C,C,2025-02-06,2027-04-15,2026-12-04,2028-02-13,2020-11-28,644.205482
3,True,True,False,False,True,771,112,569,648,380,...,C,B,D,C,2025-06-02,2025-09-13,2022-07-10,2026-03-27,2026-02-13,918.191773
4,True,True,False,True,False,902,11,801,617,328,...,B,A,C,D,2024-10-02,2027-11-06,2023-12-01,2029-03-08,2022-12-22,381.335817


In [29]:
numeric_cols = df.select_dtypes(include=["number"]).columns.tolist()
categorical_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
date_cols = df.select_dtypes(include=["datetime64"]).columns.tolist()
bool_cols = df.select_dtypes(include=["bool"]).columns.tolist()
all_cols = df.columns.tolist()

In [30]:
# def convert_categorical_to_numeric(df, cat_cols):
#     """
#     Converts categorical columns to numeric using one-hot encoding.

#     Parameters:
#     - df: pd.DataFrame, input DataFrame
#     - cat_cols: list of str, categorical columns to convert

#     Returns:
#     - df: pd.DataFrame, DataFrame with categorical columns converted to numeric
#     """
#     for col in cat_cols:
#         dummies = pd.get_dummies(df[col], prefix=col)
#         print(dummies.head())
#         break
#         # df = pd.concat([df, dummies], axis=1)
#         # df.drop(col, axis=1, inplace=True)
#     return df

# Convert categorical columns to numeric using Label Encoding
def convert_categorical_to_numeric(df, cat_cols):
    """
    Converts categorical columns to numeric using Label Encoding.

    Parameters:
    - df: pd.DataFrame, input DataFrame
    - cat_cols: list of str, categorical columns to convert

    Returns:
    - df: pd.DataFrame, DataFrame with categorical columns converted to numeric
    """
    for col in cat_cols:
        df[col] = df[col].cat.codes
    return df

convert_categorical_to_numeric(df, df.select_dtypes(include=["category"]).columns.tolist())

,bool_col_0,bool_col_1,bool_col_2,bool_col_3,bool_col_4,int_col_0,int_col_1,int_col_2,int_col_3,int_col_4,...,cat_col_1,cat_col_2,cat_col_3,cat_col_4,date_col_0,date_col_1,date_col_2,date_col_3,date_col_4,target
0,True,False,True,False,False,946,839,945,801,955,...,2,3,1,1,2025-08-29,2020-05-08,2028-12-02,2025-11-30,2029-12-21,12.137458
1,False,True,False,False,True,686,123,70,588,599,...,0,3,0,0,2027-09-07,2024-01-10,2025-05-17,2026-10-08,2020-09-21,488.643897
2,True,True,False,False,True,750,992,363,809,755,...,2,0,2,2,2025-02-06,2027-04-15,2026-12-04,2028-02-13,2020-11-28,644.205482
3,True,True,False,False,True,771,112,569,648,380,...,2,1,3,2,2025-06-02,2025-09-13,2022-07-10,2026-03-27,2026-02-13,918.191773
4,True,True,False,True,False,902,11,801,617,328,...,1,0,2,3,2024-10-02,2027-11-06,2023-12-01,2029-03-08,2022-12-22,381.335817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,True,True,True,False,True,594,924,277,286,840,...,3,3,3,0,2022-08-07,2026-06-29,2024-09-18,2029-09-02,2020-02-11,522.289473
996,True,False,True,False,False,178,914,953,9,1,...,0,0,0,0,2029-08-09,2023-05-14,2022-11-28,2023-03-07,2028-03-13,359.973518
997,False,True,True,False,True,209,493,835,541,519,...,0,2,3,0,2020-07-25,2029-01-18,2023-01-11,2024-05-16,2028-10-23,16.036400
998,False,True,False,False,False,499,522,988,755,761,...,1,3,2,0,2028-07-17,2025-05-15,2024-09-26,2027-07-19,2025-10-14,562.448941


In [31]:
# Convert text columns to numeric using TF-IDF Vectorization
from sklearn.feature_extraction.text import TfidfVectorizer

def convert_text_to_numeric(df, text_cols):
    """
    Converts text columns to numeric using TF-IDF Vectorization.

    Parameters:
    - df: pd.DataFrame, input DataFrame
    - text_cols: list of str, text columns to convert

    Returns:
    - df: pd.DataFrame, DataFrame with text columns converted to numeric
    """
    for col in text_cols:
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(df[col])
        tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
        df[col] = tfidf_matrix.toarray()
    return df

df = convert_text_to_numeric(df, df.select_dtypes(include=["object"]).columns.tolist())

In [22]:
# Convert date columns to numeric using timestamp
def convert_date_to_numeric(df, date_cols):
    """
    Converts date columns to numeric using timestamp.

    Parameters:
    - df: pd.DataFrame, input DataFrame
    - date_cols: list of str, date columns to convert

    Returns:
    - df: pd.DataFrame, DataFrame with date columns converted to numeric
    """
    for col in date_cols:
        df[col] = pd.to_datetime(df[col]).astype(np.int64) // 10**9  # Convert to seconds since epoch
        # print(f"Converted {col}: {df[col].head()}")
    return df

df = convert_date_to_numeric(df, df.select_dtypes(include=["datetime64"]).columns.tolist())

In [6]:
def convert_bool_to_numeric(df, bool_cols):
    """
    Converts boolean columns to numeric.

    Parameters:
    - df: pd.DataFrame, input DataFrame
    - bool_cols: list of str, boolean columns to convert

    Returns:
    - df: pd.DataFrame, DataFrame with boolean columns converted to numeric
    """
    for col in bool_cols:
        df[col] = df[col].astype(int)
        # print(f"Converted {col}: {df[col].head()}")
    return df

df = convert_bool_to_numeric(df, df.select_dtypes(include=["bool"]).columns.tolist())

In [7]:
# Convert numeric columns to float
def convert_numeric_to_float(df, numeric_cols):
    """
    Converts numeric columns to float.

    Parameters:
    - df: pd.DataFrame, input DataFrame
    - numeric_cols: list of str, numeric columns to convert

    Returns:
    - df: pd.DataFrame, DataFrame with numeric columns converted to float
    """
    for col in numeric_cols:
        df[col] = df[col].astype(float)
        # print(f"Converted {col}: {df[col].head()}")
    return df

df = convert_numeric_to_float(df, df.select_dtypes(include=["number"]).columns.tolist())

In [23]:
df.head()

,bool_col_0,bool_col_1,bool_col_2,bool_col_3,bool_col_4,int_col_0,int_col_1,int_col_2,int_col_3,int_col_4,...,zoofteypbp,zpknytqqcx,zqxpisjdug,zsepfdfxlj,zsmiapaape,zugevzqtvu,zulpdyewqn,zumsehnlyz,zuwpexkkrn,zyuawekjoh
0,True,False,True,False,True,946.0,839.0,945.0,801.0,955.0,...,False,False,False,False,False,False,False,False,False,False
1,False,True,False,False,True,686.0,123.0,70.0,588.0,599.0,...,False,False,False,False,False,False,False,False,False,False
2,True,True,False,False,True,750.0,992.0,236.0,809.0,755.0,...,False,False,False,False,False,False,False,False,False,False
3,True,True,False,False,True,771.0,112.0,569.0,648.0,380.0,...,False,False,False,False,False,False,False,False,False,False
4,True,True,False,True,False,902.0,11.0,801.0,617.0,328.0,...,False,False,False,False,False,False,False,False,False,False


In [9]:
def introduce_missing_values(df, missing_frac=0.1, seed=42):
    """
    Introduces missing values randomly in the DataFrame.

    Parameters:
    - df: pd.DataFrame
    - missing_frac: float, fraction of total entries to be set as NaN (default 10%)
    - seed: int, for reproducibility

    Returns:
    - pd.DataFrame with missing values
    """
    np.random.seed(seed)
    df_copy = df.copy()
    total_cells = np.product(df_copy.shape)
    num_missing = int(total_cells * missing_frac)

    # Randomly choose cells to set to NaN
    for _ in range(num_missing):
        i = np.random.randint(0, df_copy.shape[0])
        j = np.random.randint(0, df_copy.shape[1])
        df_copy.iat[i, j] = np.nan

    return df_copy

In [10]:
df_with_missing = introduce_missing_values(df, missing_frac=0.1)  # 10% of data becomes NaN

C:\Users\varun\AppData\Local\Temp\ipykernel_4048\277907646.py:1: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  df_with_missing = introduce_missing_values(df, missing_frac=0.1)  # 10% of data becomes NaN


In [11]:
def drop_duplicates(df):
    """Remove duplicate rows from the DataFrame."""
    return df.drop_duplicates()

def drop_constant_columns(df):
    """Remove columns with only a single unique value."""
    return df.loc[:, df.apply(lambda col: col.nunique(dropna=False) > 1)]

def drop_columns_by_null_threshold(df, threshold=0.5):
    """
    Drop columns with missing values above the given threshold.
    - threshold (float): Max allowable proportion of nulls (0 to 1).
    """
    null_fraction = df.isnull().mean()
    cols_to_keep = null_fraction[null_fraction <= threshold].index
    return df[cols_to_keep]

def impute_missing_values(df, strategy="mean"):
    """
    Fill missing values in numeric and categorical columns.
    - strategy (str): 'mean', 'median', or 'mode'.
    """
    df_clean = df.copy()
    
    for col in df.columns:
        if df[col].isnull().sum() == 0:
            continue
        
        if pd.api.types.is_numeric_dtype(df[col]):
            if strategy == "mean":
                fill_value = df[col].mean()
            elif strategy == "median":
                fill_value = df[col].median()
            elif strategy == "mode":
                fill_value = df[col].mode()[0]
            else:
                raise ValueError("strategy must be one of: 'mean', 'median', 'mode'")
            df_clean[col] = df[col].fillna(fill_value)
        else:
            # Categorical or text-based column: use mode only
            fill_value = df[col].mode()[0]
            df_clean[col] = df[col].fillna(fill_value)
    
    return df_clean

def strip_whitespace_string_columns(df):
    """Trim whitespace in string or object-type columns."""
    df_clean = df.copy()
    str_cols = df_clean.select_dtypes(include=["object", "string"]).columns
    for col in str_cols:
        df_clean[col] = df_clean[col].astype(str).str.strip()
    return df_clean

In [12]:
impute_missing_values.__doc__ 

"\n    Fill missing values in numeric and categorical columns.\n    - strategy (str): 'mean', 'median', or 'mode'.\n    "

In [13]:
def identify_missing_values(df):
    """
    Summarizes missing values in a DataFrame.

    Parameters:
    - df (pd.DataFrame): The input DataFrame.

    Returns:
    - pd.DataFrame: A summary table with columns, missing counts, and percentages.
    """
    missing_summary = df.isnull().sum().reset_index()
    missing_summary.columns = ['column', 'missing_count']
    missing_summary['missing_percent'] = 100 * missing_summary['missing_count'] / len(df)
    missing_summary = missing_summary[missing_summary['missing_count'] > 0]
    missing_summary = missing_summary.sort_values(by='missing_percent', ascending=False).reset_index(drop=True)
    
    return missing_summary

In [14]:
identify_missing_values(df_with_missing)

,column,missing_count,missing_percent
0,zbwfyjynfw,129,12.9
1,gecbyckzcp,129,12.9
2,bjjuexrher,127,12.7
3,jspqzazjfa,126,12.6
4,ptjlrrvskv,124,12.4
...,...,...,...
5021,pojqnnopwg,67,6.7
5022,higsybtmrd,67,6.7
5023,mqzybaenge,65,6.5
5024,wzojysntlo,64,6.4


In [15]:
df = drop_duplicates(df)
df = drop_constant_columns(df)
df = drop_columns_by_null_threshold(df, threshold=0.4)
df = impute_missing_values(df_with_missing, strategy="mode")
df = strip_whitespace_string_columns(df)

In [16]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

def normalize_minmax(df, columns=None):
    """Applies Min-Max normalization to selected numeric columns."""
    df_norm = df.copy()
    if columns is None:
        columns = df.select_dtypes(include=[np.number]).columns
    scaler = MinMaxScaler()
    df_norm[columns] = scaler.fit_transform(df_norm[columns])
    return df_norm

def standardize_zscore(df, columns=None):
    """Standardizes selected numeric columns using Z-score (mean=0, std=1)."""
    df_scaled = df.copy()
    if columns is None:
        columns = df.select_dtypes(include=[np.number]).columns
    scaler = StandardScaler()
    df_scaled[columns] = scaler.fit_transform(df_scaled[columns])
    return df_scaled

def robust_scale(df, columns=None):
    """Applies robust scaling using median and IQR (resistant to outliers)."""
    df_robust = df.copy()
    if columns is None:
        columns = df.select_dtypes(include=[np.number]).columns
    scaler = RobustScaler()
    df_robust[columns] = scaler.fit_transform(df_robust[columns])
    return df_robust

def log_transform(df, columns=None, add_constant=True):
    """
    Applies log transformation to selected numeric columns.
    - add_constant: Add 1 to avoid log(0) if True.
    """
    df_log = df.copy()
    if columns is None:
        columns = df.select_dtypes(include=[np.number]).columns
    for col in columns:
        try:
            if add_constant:
                df_log[col] = np.log1p(df_log[col])  # log(1 + x)
            else:
                df_log[col] = np.log(df_log[col])
        except Exception as e:
            print(f"Skipping log transform on {col} due to error: {e}")
    return df_log

In [17]:
numeric_cols = df.select_dtypes(include=["number"]).columns.tolist()
categorical_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
date_cols = df.select_dtypes(include=["datetime64"]).columns.tolist()
bool_cols = df.select_dtypes(include=["bool"]).columns.tolist()
all_cols = df.columns.tolist()

In [18]:
df_minmax = normalize_minmax(df, columns=numeric_cols)
df_zscore = standardize_zscore(df, columns=numeric_cols)
df_robust = robust_scale(df, columns=numeric_cols)
df_log = log_transform(df, columns=numeric_cols, add_constant=True)
df_log_no_const = log_transform(df, columns=numeric_cols, add_constant=False)

c:\Users\varun\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [19]:
def convert_str_to_numeric(df, errors='coerce'):
    """
    Converts columns with strings that look like numbers into numeric dtype.
    `errors='coerce'` will convert invalid parsing to NaN.
    """
    df_converted = df.copy()
    for col in df_converted.columns:
        if df_converted[col].dtype == object:
            try:
                df_converted[col] = pd.to_numeric(df_converted[col], errors=errors)
            except:
                pass
    return df_converted

def lowercase_string_columns(df):
    """
    Lowercases all string (object) column values.
    """
    df_cleaned = df.copy()
    str_cols = df_cleaned.select_dtypes(include='object').columns
    for col in str_cols:
        df_cleaned[col] = df_cleaned[col].astype(str).str.lower().str.strip()
    return df_cleaned

def handle_boolean_columns(df):
    """
    Converts columns with boolean-looking strings or integers into proper bool dtype.
    """
    df_bool = df.copy()
    for col in df_bool.columns:
        if df_bool[col].dtype == object:
            unique_vals = df_bool[col].dropna().astype(str).str.lower().unique()
            if set(unique_vals).issubset({'true', 'false', 'yes', 'no', '1', '0'}):
                df_bool[col] = df_bool[col].astype(str).str.lower().map({
                    'true': True, '1': True, 'yes': True,
                    'false': False, '0': False, 'no': False
                })
        elif df_bool[col].dtype in [int, float] and set(df_bool[col].dropna().unique()).issubset({0, 1}):
            df_bool[col] = df_bool[col].astype(bool)
    return df_bool


In [20]:
# df = convert_str_to_numeric(df, errors='coerce')
df = convert_str_to_numeric(df, errors='ignore')
# df = convert_str_to_numeric(df, errors='raise')
df = lowercase_string_columns(df)
df = handle_boolean_columns(df)

In [21]:
df.head()

,bool_col_0,bool_col_1,bool_col_2,bool_col_3,bool_col_4,int_col_0,int_col_1,int_col_2,int_col_3,int_col_4,...,zoofteypbp,zpknytqqcx,zqxpisjdug,zsepfdfxlj,zsmiapaape,zugevzqtvu,zulpdyewqn,zumsehnlyz,zuwpexkkrn,zyuawekjoh
0,True,False,True,False,True,946.0,839.0,945.0,801.0,955.0,...,False,False,False,False,False,False,False,False,False,False
1,False,True,False,False,True,686.0,123.0,70.0,588.0,599.0,...,False,False,False,False,False,False,False,False,False,False
2,True,True,False,False,True,750.0,992.0,236.0,809.0,755.0,...,False,False,False,False,False,False,False,False,False,False
3,True,True,False,False,True,771.0,112.0,569.0,648.0,380.0,...,False,False,False,False,False,False,False,False,False,False
4,True,True,False,True,False,902.0,11.0,801.0,617.0,328.0,...,False,False,False,False,False,False,False,False,False,False
